In [1]:
import os
import time
from PyPDF2 import PdfReader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_pinecone import PineconeVectorStore
from pinecone import Pinecone, ServerlessSpec
from langchain_classic.chains import create_retrieval_chain 
from langchain_classic.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
from dotenv import load_dotenv
from langchain_core.documents import Document
from uuid import uuid4
from langchain_cohere import CohereRerank
from langchain_classic.retrievers import ContextualCompressionRetriever
from langchain_classic.retrievers.multi_query import MultiQueryRetriever

/Users/shenglienlee/anaconda3/envs/Me_RAG/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Create a new RAG from Scratch(if the DB is not created use this)

In [ ]:
#loading and chunk the pdf file
def load_and_chunk_pdfs(file_paths):
    documents = [] 
    #chunk pdf one by one(if there are more than one PDF)
    for path in file_paths:
        single_file_text = ""
        try:
            reader = PdfReader(path)
            #extract text from each page of pdf and combine them together
            for i, page in enumerate(reader.pages):
                text = page.extract_text()
                if text:
                    single_file_text  = single_file_text + text + " "
        except Exception as e:
            print(f"   can't read the file, {path}: {e}")
            continue

        #chunk the pdf
        text_splitter = RecursiveCharacterTextSplitter(
            chunk_size=1000, 
            chunk_overlap=400,
            separators=["\n\n", "\n", ".", "?", "!", ";"],
            is_separator_regex=False 
        )
        page_chunk = text_splitter.split_text(single_file_text)        

        #transfer each chunk to a document object
        for chunk in page_chunk:
            doc = Document(
                page_content=chunk, 
                #info of the chunk, such as source, page...
                metadata={           
                    "Source": "The Almanack of Naval Ravikant",
                    "page":i+1,
                    "type": "book"
                }
            )
            documents.append(doc)

        print(f"   -> {path} created {len(page_chunk)} Document objects。")

    print(f"Finished. Total {len(documents)} objects.")
    return documents

#init the pinecone and upload documents
def init_vector_store(documents):
    #init the Embedding model
    embeddings = OpenAIEmbeddings(model=EMBEDDING_MODEL)

    # init the pinecone
    pc = Pinecone(api_key=os.environ.get("PINECONE_API_KEY"))

    #check if the index exist if not create one.
    existing_indexes = [i.name for i in pc.list_indexes()]
    if INDEX_NAME not in existing_indexes:
        print(f"Creating Pinecone Index: {INDEX_NAME} (Serverless)...")
        pc.create_index(
            name=INDEX_NAME,
            dimension=DIMENSION,
            metric="cosine",
            spec=ServerlessSpec(cloud="aws", region="us-east-1")
        )
        #wait for initialization
        while not pc.describe_index(INDEX_NAME).status.ready:
            time.sleep(1)
    index = pc.Index(INDEX_NAME)

    print("uploading embeddings.")
    vector_store = PineconeVectorStore(index=index, embedding=embeddings)
    #create unique id for each chunk
    uuids = [str(uuid4()) for _ in range(len(documents))]
    vector_store.add_documents(documents=documents, ids=uuids)
    print(f"Index {INDEX_NAME} is set.")

    return vector_store

#create the rag chain
def setup_rag_chain(vector_store):
    
    #set the LLM
    llm = ChatOpenAI(model="gpt-4o", temperature=0.1)

    # set pinecone retriever
    # I use cosine si,ilarity to calculate the distnace between each vector.
    base_retriever = vector_store.as_retriever(
        #Cosine similarity
        search_type="similarity",
        #Return Top 20 closest results. 
        search_kwargs={"k":15} 
    )

    # MultiQuery Retriever -> exapnd the user query to have more accurate result.
    multi_query_retriever = MultiQueryRetriever.from_llm(
        retriever=base_retriever,
        llm=llm
    )

    # Rerank by Cohere's cross encoder
    compressor = CohereRerank(
        #the model for english
        model="rerank-english-v3.0", 
        #rerank and get the top 5 results
        top_n=5                     
    )

    #Combine the pinecone_retriver and reranking
    rerank_retriever = ContextualCompressionRetriever(
        base_compressor=compressor,
        base_retriever=multi_query_retriever
    )

    system_prompt = (
        "You are an AI embodiment of **Naval Ravikant** based on his Almanack.\n"
        "Your goal is to provide the best advice that Naval most probably will provide to the people who ask you question.\n"
        "Style: Direct, concise, rational, and first-principles thinking.\n"
        "Instructions:\n"
        "1. Answer strictly based on the provided book context.\n"
        "2. If applicable, mention the page number from the metadata (e.g., [Page 42]).\n"
        "3. Use bullet points for lists, as Naval often does.\n"
        "4. Answer in Traditional Chinese.\n"
        "Context: {context}"
    )
    prompt = ChatPromptTemplate.from_messages([
        ("system", system_prompt),
        ("human", "{input}"),
    ])

    #stuff the prompt to LLM
    combine_docs_chain = create_stuff_documents_chain(llm, prompt)
    #chain the rerank retriever and combine_docs_chain
    rag_chain = create_retrieval_chain(rerank_retriever, combine_docs_chain)

    return rag_chain

# --- 主程式執行區 ---
if __name__ == "__main__":
    # Load the .env file
    load_dotenv()
    # 參數設定
    INDEX_NAME = "the-almanack-of-naval-ravikant-rag"
    EMBEDDING_MODEL = "text-embedding-3-large"
    DIMENSION = 3072
    # 指定檔案路徑
    path = r"your path"
    pdf_files = [path]
    
    # 1. read and chunk PDFs
    documents = load_and_chunk_pdfs(pdf_files)
    
    # 2. prepare the vector db
    vector_store = init_vector_store(documents)
    
    # 3. create the Rag chain
    rag_chain = setup_rag_chain(vector_store)
    
    # 4. test
    print("\n" + "="*30)
    print("RAG Bot is ready (enter 'exit' to end the chat)")
    print("="*30)
    
    while True:
        query = input("\n your query: ")
        if query.lower() in ["exit", "quit"]:
            break
            
        print("Thinking...")
        response = rag_chain.invoke({"input": query})
        
        print(f"\n Answer:\n{response['answer']}")
        
        # #print references
        # print("\n🔍 referemce (Context):")
        # for i, doc in enumerate(response["context"]):
        #     print(f"--- Chunk {i+1} ---")
        #     print(doc.page_content[:100] + "...")

   -> /Users/shenglienlee/Documents/GitHub/quick-start-guide-to-llms?tab=readme-ov-file/notebooks/My_project/doc/Eric-Jorgenson_The-Almanack-of-Naval-Ravikant_Final.pdf created 451 Document objects。
Finished. Total 451 objects.
Creating Pinecone Index: the-almanack-of-naval-ravikant-rag (Serverless)...
uploading embeddings.
Index the-almanack-of-naval-ravikant-rag is set.

RAG Bot is ready (enter 'exit' to end the chat)
Thinking...

 Answer:
要擁有更好的人生，可以考慮以下幾點：

- **選擇幸福**：幸福是一種選擇和技能。你可以通過改變內心來提升幸福感，而不是依賴外部環境的改變。[Page 134]
- **培養良好習慣**：用好的習慣取代壞習慣，並與積極向上的人交往。你的習慣和你周圍的人會影響你的長期幸福。[Page 140]
- **內心的平和**：追求內心的平和，而不是試圖解決所有外部問題。放下對問題的執著，才能獲得內心的平靜。[Page 143]
- **降低身份認同**：減少對不重要事物的關注，不參與政治，遠離不快樂的人，珍惜生命中的時間。[Page 133]
- **活在當下**：不要過多地擔心未來或過去，接受當下的現實，這樣才能找到內心的平和。[Page 4]
- **避免嫉妒**：嫉妒是幸福的敵人，專注於自己的生活，而不是與他人比較。[Page 77]

這些方法可以幫助你逐步提升生活質量，並找到內心的平和與幸福。


## About multiquery Retriever
假設您設定 search_kwargs={"k": 20}，且 MultiQuery 生成了 3 個問題：  
生成問題： GPT-4o 產生 Q1, Q2, Q3。  
分別搜尋：Q1 -> 去 Pinecone 抓 20 筆。  
Q2 -> 去 Pinecone 抓 20 筆。  
Q3 -> 去 Pinecone 抓 20 筆。  
原始總數： 總共抓回 $20 + 20 + 20 = 60$ 筆資料。  
關鍵步驟 - 去重 (Deduplication)：  
LangChain 會自動執行 「取聯集 (Union)」 的動作。情境： 如果「第 42 頁的某一段」在 Q1 被抓到，在 Q2 也被抓到。  
結果： 它只會算 1 筆，不會重複。  
最終送給 Reranker 的數量：  
最少： 20 筆 (如果 3 個問題抓回來的完全一模一樣)。  
最多： 60 筆 (如果 3 個問題抓回來的完全不重疊)。平均經驗值： 通常會落在 35 ~ 50 筆 之間。

# use exist RAG

In [ ]:
def setup_rag_chain(vector_store):
    llm = ChatOpenAI(model="gpt-4o", temperature=0.1)
    
# set pinecone retriever
    # I use cosine si,ilarity to calculate the distnace between each vector.
    base_retriever = vector_store.as_retriever(
        #Cosine similarity
        search_type="similarity",
        #Return Top 20 closest results. 
        search_kwargs={"k":15} 
    )

    # MultiQuery Retriever -> exapnd the user query to have more accurate result.
    multi_query_retriever = MultiQueryRetriever.from_llm(
        retriever=base_retriever,
        llm=llm
    )

    # Rerank by Cohere's cross encoder
    compressor = CohereRerank(
        #the model for english
        model="rerank-english-v3.0", 
        #rerank and get the top 5 results
        top_n=5                     
    )

    #Combine the pinecone_retriver and reranking
    rerank_retriever = ContextualCompressionRetriever(
        base_compressor=compressor,
        base_retriever=multi_query_retriever
    )

    system_prompt = (
        "You are an AI embodiment of Naval Ravikant based on his Almanack.\n"
        "Your goal is to provide the best advice that Naval most probably will provide to the people who ask you question.\n"
        "Style: Direct, concise, rational, and first-principles thinking.\n"
        "Instructions:\n"
        "1. Answer strictly based on the provided book context.\n"
        "2. If applicable, mention the page number from the metadata (e.g., [Page 42]).\n"
        "3. Use bullet points for lists, as Naval often does.\n"
        "4. Answer in Traditional Chinese.\n"
        "Context: {context}"
    )

    prompt = ChatPromptTemplate.from_messages([
        ("system", system_prompt),
        ("human", "{input}"),
    ])


    combine_docs_chain = create_stuff_documents_chain(llm, prompt)

    rag_chain = create_retrieval_chain(rerank_retriever, combine_docs_chain)

    return rag_chain


if __name__ == "__main__":
    # Load the .env file
    load_dotenv()

    # Retrieve the Pinecone API key from user data
    pinecone_key = os.environ.get('PINECONE_API_KEY')

    # Initialize the OpenAI client with the API key from user data
    OpenAI_key=os.environ.get("OPENAI_API_KEY")




    INDEX_NAME = "the-almanack-of-naval-ravikant-rag"
    EMBEDDING_MODEL = "text-embedding-3-large"
    DIMENSION = 3072
    path = r"/Users/shenglienlee/Documents/GitHub/quick-start-guide-to-llms?tab=readme-ov-file/notebooks/My_project/doc/Eric-Jorgenson_The-Almanack-of-Naval-Ravikant_Final.pdf"
    pdf_files = [path]
    
    embeddings = OpenAIEmbeddings(model="text-embedding-3-large")
    pc = Pinecone(api_key=os.environ.get("PINECONE_API_KEY"))
    vector_store = PineconeVectorStore.from_existing_index(
        index_name=INDEX_NAME,
        embedding=embeddings
    )
    
    rag_chain = setup_rag_chain(vector_store)
    
    #test
    print("\n" + "="*30)
    print("RAG Bot is ready (enter 'exit' to end the chat)")
    print("="*30)
    
    while True:
        query = input("\n your query: ")
        if query.lower() in ["exit", "quit"]:
            break
        print(f"User: {query}")
        print("Thinking...")
        response = rag_chain.invoke({"input": query})
        
        print(f"\n Answer:\n{response['answer']}")
        

        # print("\n🔍 Reference: (Context):")
        # for i, doc in enumerate(response["context"]):
        #     print(f"--- Chunk {i+1} ---")
        #     print(doc.page_content[:100] + "...") 


RAG Bot is ready (enter 'exit' to end the chat)
User: how to have a good lif
Thinking...

 Answer:
要過上好生活，可以考慮以下幾點：

- **建立良好習慣**：戒酒、少吃糖、不沉迷於社交媒體，這些都能讓你的情緒更穩定。[Page 147]
- **選擇積極的社交圈**：與積極向上、低維護的朋友交往，這樣的關係能提升你的幸福感。[Page 147]
- **學會接受**：在生活中，你總有三個選擇：改變它、接受它或離開它。接受現實能讓你更快適應。[Page 152]
- **保持當下的存在感**：避免過多地活在過去或未來，這樣能讓你更快樂。[Page 135]
- **關注健康**：優先考慮自己的身體、心理和精神健康，這是其他一切的基礎。[Page 160]

這些建議能幫助你在生活中找到平衡和幸福。


# Test

In [10]:
path = r"/Users/shenglienlee/Documents/GitHub/quick-start-guide-to-llms?tab=readme-ov-file/notebooks/My_project/doc/Eric-Jorgenson_The-Almanack-of-Naval-Ravikant_Final.pdf"
file_paths = [path]
documents = [] 
for path in file_paths:
    single_file_text = ""
    try:
        reader = PdfReader(path)
        #extract text from each page of pdf and combine them together
        for i, page in enumerate(reader.pages):
            text = page.extract_text()
            if text:
                single_file_text  = single_file_text + text + " "
    except Exception as e:
        print(f"   ⚠️ 無法讀取 {path}: {e}")
        continue
    #chunk the pdf
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=2500, # 設定為 2500，約涵蓋 2-3 個段落
        chunk_overlap=400, # 400 字重疊，確保跨段落語意
        separators=["\n \n", "\n", "。", "！", "？", " ", ""],
        is_separator_regex=False 
    )
    
    page_chunk = text_splitter.split_text(single_file_text)
    
    # ----------------
    
    # --- 3. 包裝成 Document 物件 (關鍵修改) ---
    
    for chunk in page_chunk:
        # 建立 Document 物件
        doc = Document(
            page_content=chunk,  # 👈 內文保持乾淨 (Clean Text)
            metadata={           # 👈 來源資訊藏在這裡
                "Source": "The Almanack of Naval Ravikant",
                "page":i+1,
                "type": "book"
            }
        )
        documents.append(doc)

    print(f"   -> {path} 建立了 {len(page_chunk)} 個 Document 物件。")
print(f"✅ 所有文件處理完畢，共 {len(documents)} 個物件。")

   -> /Users/shenglienlee/Documents/GitHub/quick-start-guide-to-llms?tab=readme-ov-file/notebooks/My_project/doc/Eric-Jorgenson_The-Almanack-of-Naval-Ravikant_Final.pdf 建立了 130 個 Document 物件。
✅ 所有文件處理完畢，共 130 個物件。


In [8]:
PdfReader(path)

In [9]:
for i, page in enumerate(reader.pages):
    text = page.extract_text()
    print(text)

THE ALMANACK OF NAVAL RAVIKANT

GETTING RICH IS NOT JUST  
ABOUT LUCK; HAPPINESS IS  
NOT JUST A TRAIT WE ARE  
BORN WITH.
These aspirations may seem out of reach, but building wealth and be -
ing happy are skills we can learn.
So what are these skills, and how do we learn them? What are the 
principles that should guide our efforts? What does progress really 
look like?
Naval Ravikant is an entrepreneur, philosopher, and investor who 
has captivated the world with his principles for building wealth and 
creating long-term happiness. The Almanack of Naval Ravikant  is a  
collection of Naval’s wisdom and experience from the last ten years, 
shared as a curation of his most insightful interviews and poignant 
reflections. This isn’t a how-to book, or a step-by-step gimmick. In -
stead, through Naval’s own words, you will learn how to walk your own 
unique path toward a happier, wealthier life.
This book has been created as a public service. It is available for free 
download in pdf and 